# Скрипт по сбору данных

## Инициализация скрипта

In [18]:
#1
# Loading Libraries

import pandas as pd
import googleapiclient.discovery as api
import numpy as np
import requests

from pathlib import Path
import os
import yaml
import json

In [19]:
#2

# Defining variables

# Determine path to the working directory
path = Path().parent.resolve()
print('Current Path')
print(path)

# Opening and parsing yaml file

cnf_path = str(path) + '/' + 'cnf.yaml'

with open(cnf_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

Current Path
/Users/andrei/PycharmProjects/DM_Games_Data


In [20]:
#3

# Загрузка масок для запросов

with open('queries_iter_channels.json', 'r', encoding='utf-8') as f:
    queries = json.load(f)

In [21]:
# 4

game_names = queries['games']
print('There are game names to parse', '\n', game_names)
print()
publishers = queries['publishers']
print('There are publisher names to parse''\n',publishers)

There are game names to parse 
 ['8-Bit Armies', "Assassin's Creed Odyssey", "Assassin's Creed Origins", 'Battle Chasers: Nightwar', 'Battleborn', 'Battleborn', 'Battlefield 1', 'Battlefield V', 'Bayonetta', 'Beat Cop', 'Bomber Crew', 'Bomber Crew', 'Call of Duty: Infinite Warfare', 'Call of Duty: WWII', 'Conan Exiles', 'Crashlands', 'Cuphead', 'Danganronpa: Trigger Happy Havoc', 'Dark Souls III', 'Darkest Dungeon', 'Darksiders III', 'Dead Cells', 'Dead Rising 4', 'Deponia Doomsday', 'Destiny 2', 'Destiny 2: Forsaken', 'Deus Ex: Mankind Divided', 'Deus Ex: Mankind Divided', 'Devil Daggers', 'DiRT 4', 'Dishonored 2', 'Dishonored: Death of the Outsider', 'Dragon Ball FighterZ', "Dragon's Dogma: Dark Arisen", 'Dungeon Souls', 'Dungeons 3', 'Dying Light', 'Elex', 'Enter the Gungeon', 'F1 2016', 'F1 2017', 'F1 2017', 'F1 2018', 'FIFA 17', 'Fallout 4 VR', 'Far Cry 5', 'Far Cry: Primal', 'Farming Simulator 17', 'Farming Simulator 19', 'Farming Simulator 19', 'Final Fantasy X / X-2 HD Remaster

# Создание клиента запроса

In [22]:
# 6

# Создание обертки над API для работы с поисковыми масками

class YoutubeDataCollector(object):

    def __init__(self, cl,q_params):
        self.params = q_params
        self.client = cl

    def make_request(self, query):
        request = self.client.search().list(q = query,
        **self.params)
        try:
            return request.execute()
        except:
            return None

    def make_iter_request(self, iter_list, add_mask = ''):
        """

        :param iter_list:
        :param add_mask:
        :return:
        """

        data = pd.DataFrame()

        for iter in iter_list:

            q = iter + add_mask

            response = self.make_request(q)
            if response:

                tmp = pd.json_normalize(response['items'])
                data = pd.concat([data,tmp])
                continue

            else:
                print('Empty_response')

        return data

    def make_page_request(self):
        pass

# Базовый запрос

In [23]:
# 7

mask_review = ' & обзор'
mask_play =  ' & (прохождение | летсплей | play)'


In [29]:
# Издательства и их каналы

api_b = config['api_client']
query_params = config['QueryParams']['channels']

# Для издательств лимит по результатам будет достаточен в 10
query_params['maxResults']= 10

In [26]:
youtube = api.build(**api_b)

collector = YoutubeDataCollector(youtube,query_params)

In [27]:


response_df = collector.make_iter_request(publishers[:4], add_mask = '')

In [28]:
response_df

,kind,etag,id.kind,id.channelId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.medium.url,snippet.thumbnails.high.url,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,aYP0SmeDvFVNqbwyfFdAziq0mb4,youtube#channel,UCq2-YS92kRURomhlw2fj7_g,2010-12-06T12:34:20Z,UCq2-YS92kRURomhlw2fj7_g,11 bit studios,This is official channel of 11 bit studios. He...,https://yt3.ggpht.com/ytc/AKedOLQW4-g5X6xj1402...,https://yt3.ggpht.com/ytc/AKedOLQW4-g5X6xj1402...,https://yt3.ggpht.com/ytc/AKedOLQW4-g5X6xj1402...,11 bit studios,upcoming,2010-12-06T12:34:20Z
1,youtube#searchResult,S75cZREShyVn5xE643zQuiB6HnI,youtube#channel,UCxQYaOzUx8l5I8BTipZhqmw,2012-02-22T23:01:08Z,UCxQYaOzUx8l5I8BTipZhqmw,LetsPlayPoRusski,"Обзоры, прохождения, критика различных игр. Вс...",https://yt3.ggpht.com/ytc/AKedOLRq2rDr5po1JTQI...,https://yt3.ggpht.com/ytc/AKedOLRq2rDr5po1JTQI...,https://yt3.ggpht.com/ytc/AKedOLRq2rDr5po1JTQI...,LetsPlayPoRusski,upcoming,2012-02-22T23:01:08Z
2,youtube#searchResult,XeN4P5JEq_q_krPb4iyjRNAECWU,youtube#channel,UCeUgIQ-qNv8SabDbPuaPtgQ,2017-02-08T10:13:38Z,UCeUgIQ-qNv8SabDbPuaPtgQ,BlackKazan,Добро пожаловать На канале Инди Игр. На этом к...,https://yt3.ggpht.com/ytc/AKedOLRohPcGN5v-vzXo...,https://yt3.ggpht.com/ytc/AKedOLRohPcGN5v-vzXo...,https://yt3.ggpht.com/ytc/AKedOLRohPcGN5v-vzXo...,BlackKazan,none,2017-02-08T10:13:38Z
3,youtube#searchResult,6F6VoacYxXUzIX8Z8Pw4tAGzEkA,youtube#channel,UC3PkpMBNfeTxEzYhpJrVPwA,2012-05-14T19:30:47Z,UC3PkpMBNfeTxEzYhpJrVPwA,Wildzeck SHOW,"Привет, братва! Меня зовут Иван и я создаю рол...",https://yt3.ggpht.com/ytc/AKedOLT4QedGhKQef1Q2...,https://yt3.ggpht.com/ytc/AKedOLT4QedGhKQef1Q2...,https://yt3.ggpht.com/ytc/AKedOLT4QedGhKQef1Q2...,Wildzeck SHOW,none,2012-05-14T19:30:47Z
4,youtube#searchResult,CA7K2k46mFOOwyb5OqHDaEHVBv0,youtube#channel,UCG78X91v6huh5Wm6Ay0eq3g,2016-01-03T15:52:07Z,UCG78X91v6huh5Wm6Ay0eq3g,ZalonBit,Всім привіт! Мене звати Ігор мій нікнейм Zalon...,https://yt3.ggpht.com/ytc/AKedOLRjcL19up-kA-jC...,https://yt3.ggpht.com/ytc/AKedOLRjcL19up-kA-jC...,https://yt3.ggpht.com/ytc/AKedOLRjcL19up-kA-jC...,ZalonBit,upcoming,2016-01-03T15:52:07Z
5,youtube#searchResult,N3G32sK6J3uNwTk9tIn1OY7iNfo,youtube#channel,UCsSA0U5KychQochrGibyafw,2016-10-11T11:46:14Z,UCsSA0U5KychQochrGibyafw,HONEST GAME,Добро пожаловать на Honest Game - самый честны...,https://yt3.ggpht.com/ytc/AKedOLTce4f4xq6tw8rq...,https://yt3.ggpht.com/ytc/AKedOLTce4f4xq6tw8rq...,https://yt3.ggpht.com/ytc/AKedOLTce4f4xq6tw8rq...,HONEST GAME,upcoming,2016-10-11T11:46:14Z
6,youtube#searchResult,xzXW8tGoClYC2oohTTg7PooAffg,youtube#channel,UC9-9DNoVstZ-uSD_KDC0RHQ,2013-01-05T00:31:19Z,UC9-9DNoVstZ-uSD_KDC0RHQ,MoonCakesPlay,Всем привет! Меня зовут Сергей! И это канал Mo...,https://yt3.ggpht.com/ytc/AKedOLSjecO1zUxjIQNj...,https://yt3.ggpht.com/ytc/AKedOLSjecO1zUxjIQNj...,https://yt3.ggpht.com/ytc/AKedOLSjecO1zUxjIQNj...,MoonCakesPlay,none,2013-01-05T00:31:19Z
7,youtube#searchResult,RJRTmJZYMBVOgR1XAaocFn5qPgQ,youtube#channel,UCsfgc4LirS1i7HTam6DPvkg,2014-11-20T15:55:40Z,UCsfgc4LirS1i7HTam6DPvkg,Digital Sun Games,"Indie game Studio. Makers of Moonlighter, a ga...",https://yt3.ggpht.com/ytc/AKedOLTYla5nHVWzwj_n...,https://yt3.ggpht.com/ytc/AKedOLTYla5nHVWzwj_n...,https://yt3.ggpht.com/ytc/AKedOLTYla5nHVWzwj_n...,Digital Sun Games,none,2014-11-20T15:55:40Z
8,youtube#searchResult,JltyPkUaBrjJYg_9mWM31UTlzww,youtube#channel,UCR-LhAKxp44WO_ZTTwYNyIQ,2014-11-30T15:13:36Z,UCR-LhAKxp44WO_ZTTwYNyIQ,Уголок Дрима,"Добро пожаловать на игровой канал, ориентирова...",https://yt3.ggpht.com/ytc/AKedOLQFM2__gn0OBIXH...,https://yt3.ggpht.com/ytc/AKedOLQFM2__gn0OBIXH...,https://yt3.ggpht.com/ytc/AKedOLQFM2__gn0OBIXH...,Уголок Дрима,none,2014-11-30T15:13:36Z
9,youtube#searchResult,mmqhq15oy-jzxUvvfh18VaScFlo,youtube#channel,UCKfLod7MgZC9XzHix3L_vRQ,2019-05-27T20:36:15Z,UCKfLod7MgZC9XzHix3L_vRQ,Redlack 812,Oyun Spor Sohbet ve Eğlence Kanalı..,http

In [30]:
# Издания и обзоры

api_b = config['api_client']
query_params = config['QueryParams']['channels']

# Для обзоров лимит по результатам будет достаточен в 10
query_params['maxResults']= 50

youtube = api.build(**api_b)

collector = YoutubeDataCollector(youtube,query_params)


In [31]:
reviews = collector.make_iter_request(game_names[:2], add_mask = mask_review)

reviews

,kind,etag,id.kind,id.channelId,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.medium.url,snippet.thumbnails.high.url,snippet.channelTitle,snippet.liveBroadcastContent,snippet.publishTime
0,youtube#searchResult,e8P_hxMnv-Pr8GbHC6ta7hPyGPo,youtube#channel,UCu07wIgnKDrZbZkVgkhUCbg,2016-04-25T18:03:51Z,UCu07wIgnKDrZbZkVgkhUCbg,Ивангай,Ищиш летсплеи тебе сюда а также обзор игр и вс...,https://yt3.ggpht.com/ytc/AKedOLRTboygKr0-Ua-G...,https://yt3.ggpht.com/ytc/AKedOLRTboygKr0-Ua-G...,https://yt3.ggpht.com/ytc/AKedOLRTboygKr0-Ua-G...,Ивангай,upcoming,2016-04-25T18:03:51Z
1,youtube#searchResult,Yw1JT_aTay4AdNlCc1RXiIrikJw,youtube#channel,UCels-4bLVjM-Uqr6G38VPxA,2021-05-02T10:12:46Z,UCels-4bLVjM-Uqr6G38VPxA,Taco_vender Gaming,Welcome to Taco_vender Gaming! i will be doing...,https://yt3.ggpht.com/ytc/AKedOLR33uvngtbSSpOt...,https://yt3.ggpht.com/ytc/AKedOLR33uvngtbSSpOt...,https://yt3.ggpht.com/ytc/AKedOLR33uvngtbSSpOt...,Taco_vender Gaming,none,2021-05-02T10:12:46Z
2,youtube#searchResult,BPTZ5Zlnsc-Zyu0bjYzxEBaX_1M,youtube#channel,UCc-TUBptexwXN_PpgULV5eA,2017-02-03T13:24:36Z,UCc-TUBptexwXN_PpgULV5eA,Pass2Shock Productions,Иногда ты просто берешь и делаешь. Что делаешь...,https://yt3.ggpht.com/ytc/AKedOLSNPoXvfk6Wtn0e...,https://yt3.ggpht.com/ytc/AKedOLSNPoXvfk6Wtn0e...,https://yt3.ggpht.com/ytc/AKedOLSNPoXvfk6Wtn0e...,Pass2Shock Productions,none,2017-02-03T13:24:36Z
3,youtube#searchResult,nwythi0e5tKsBsGfcB6IVMfI1mA,youtube#channel,UCorkyFoPfQ6uZtl8vRyU1ew,2009-08-14T17:20:47Z,UCorkyFoPfQ6uZtl8vRyU1ew,Petroglyph Games,"Petroglyph Games, Inc. is a game studio in Las...",https://yt3.ggpht.com/ytc/AKedOLQg1ysBtRMoyoNQ...,https://yt3.ggpht.com/ytc/AKedOLQg1ysBtRMoyoNQ...,https://yt3.ggpht.com/ytc/AKedOLQg1ysBtRMoyoNQ...,Petroglyph Games,none,2009-08-14T17:20:47Z
4,youtube#searchResult,Yb8qPHiuMXRJ1ry3dWUXQM_qZLM,youtube#channel,UC1sHgkZ4Nk6vyFtteS2nGQQ,2016-07-20T06:58:27Z,UC1sHgkZ4Nk6vyFtteS2nGQQ,Toxinchik,если ты попалы на мой каналы сразу подпишись Н...,https://yt3.ggpht.com/ytc/AKedOLSeP8qjH7VQFphQ...,https://yt3.ggpht.com/ytc/AKedOLSeP8qjH7VQFphQ...,https://yt3.ggpht.com/ytc/AKedOLSeP8qjH7VQFphQ...,Toxinchik,none,2016-07-20T06:58:27Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,youtube#searchResult,I21684Tlh4W0mE6IA9ruVAXeBdE,youtube#channel,UC9xa6ikR6fw9TR-Np6DUdAQ,2010-11-03T14:59:33Z,UC9xa6ikR6fw9TR-Np6DUdAQ,PsykoloGG,,https://yt3.ggpht.com/ytc/AKedOLR9Rc5taKq3_dE8...,https://yt3.ggpht.com/ytc/AKedOLR9Rc5taKq3_dE8...,https://yt3.ggpht.com/ytc/AKedOLR9Rc5taKq3_dE8...,PsykoloGG,none,2010-11-03T14:59:33Z
46,youtube#searchResult,44q7rkIEIRY0Np0kNAQmMZZ-dOg,youtube#channel,UC15EKIQkLXKe7VlG6nvctyw,2018-12-09T13:03:41Z,UC15EKIQkLXKe7VlG6nvctyw,Зовите меня Егог,,https://yt3.ggpht.com/ytc/AKedOLR--hurpnOQm1_w...,https://yt3.ggpht.com/ytc/AKedOLR--hurpnOQm1_w...,https://yt3.ggpht.com/ytc/AKedOLR--hurpnOQm1_w...,Зовите меня Егог,none,2018-12-09T13:03:41Z
47,youtube#searchResult,zFnJ1GQKWqdgJ7H6bnJrI7CyPwg,youtube#channel,UCnGRUpCHxeDmOIG7tmozoBA,2015-03-21T21:37:45Z,UCnGRUpCHxeDmOIG7tmozoBA,Game Attention,,https://yt3.ggpht.com/ytc/AKedOLRh4sVGYxecvKUv...,https://yt3.ggpht.com/ytc/AKedOLRh4sVGYxecvKUv...,https://yt3.ggpht.com/ytc/AKedOLRh4sVGYxecvKUv...,Game Attention,upcoming,2015-03-21T21:37:45Z
48,youtube#searchResult,xYtGKddwbXChwyJUQSzYrXQMxRo,youtube#channel,UCmB08z6XksLVVj_YbsujBUg,2013-07-23T15:25:42Z,UCmB08z6XksLVVj_YbsujBUg,MyxA Tv,Приветствуем вас на нашем канале MyxA TV! Сним...,https://yt3.ggpht.com/ytc/AKedOLTDV9X7ZDCk-2cQ...,https://yt3.ggpht.com/ytc/AKedOLTDV9X7ZDCk-2cQ...,https://yt3.ggpht.com/ytc/AKedOLTDV9X7ZDCk-2cQ...,MyxA Tv,none,2013-07-23T15:25:42Z


In [ ]:
# Блогеры и их каналы

# Код для исполнителя по прохождениям